In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import baikalfunctions as bfunc
import scheme_mar2023 as scheme
import mysecure

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
# pd.set_option('display.min_rows', 100)
%matplotlib inline
plt.style.use('dark_background')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 5]
#plt.subplots_adjust(top=1, left=0, right=1, bottom=0)

In [ ]:
pwd = 'G:\\1_Data1\\95_BaikalOct2023\\'
saveImgPath = 'G:\\1_Data1\\95_BaikalOct2023\\'
saveDataPath = 'G:\\1_Data1\\95_BaikalOct2023\\'

In [ ]:
df = pd.read_csv(pwd+'origin.dat', sep='\t', parse_dates=['date_time'], dayfirst=True, header=[0], na_values='--', decimal=',')
df.info()

In [ ]:
df.head(500).tail(200).sample(5)

In [ ]:
list(df.columns)

In [ ]:
df.rename(columns=scheme.NAME_CONV_BAS, inplace=True)
df.sort_values(by='DateTime', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head(2)

In [ ]:
df.sort_values(by='DateTime', inplace=True)

In [ ]:
### remove service variables
df.drop([x for x in df.columns if x.lower().endswith('err')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().endswith('max')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().endswith('min')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().startswith('pump')], axis='columns', inplace=True, errors='ignore')
df.drop(['HumidityAir', ], axis='columns', inplace=True, errors='ignore')


In [ ]:
df.shape, df.columns

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
fig, axs = plt.subplots(7, 1)

axs[0].set_title('vCO2, ppm')
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')

axs[1].set_title('vCH4, ppm')
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')

axs[2].set_title('Temp')
axs[2].plot(df.DateTime, df['TempEqu1'], 'r-')
axs[2].plot(df.DateTime, df['TempEqu2'], 'g-')
axs[2].plot(df.DateTime, df['TempAir'], 'c-')

axs[3].set_title('WaterFlow')
axs[3].plot(df.DateTime, df['WaterFlowEqu1'], 'r-')
axs[3].plot(df.DateTime, df['WaterFlowEqu2'], 'g-')

axs[4].set_title('Press')
axs[4].plot(df.DateTime, df['PressAir'], 'c-')
axs[5].set_title('Solar')
axs[5].plot(df.DateTime, df['LightLX'], 'w-')
axs[6].set_title('SolarUV')
axs[6].plot(df.DateTime, df['LightUV'], 'b-')



### Correction data  
 ** Be carefull !!!  Make it ONCE !!! **


In [ ]:
## gether/apply specific variables
### data corrections
# df['vCO2'] = df['CO2_dry']
# df['vCH4'] = df['CH4_dry']
df['LightLX'] = df['LightLX'] * scheme.solar_cor[0] + scheme.solar_cor[1]
df['LightUV'] = df['LightUV'] * scheme.solar_uv_cor[0] + scheme.solar_uv_cor[1]
df['PressAir'] = bfunc.pressConvert(df['PressAir'])
df['PressAir'] = df['PressAir'] * scheme.press_cor[0] + scheme.press_cor[1]

### servise data corrections
df['TempEqu1'] = df['TempEqu1'] * scheme.tempEqu1_cor[0] + scheme.tempEqu1_cor[1]
df['TempEqu2'] = df['TempEqu2'] * scheme.tempEqu2_cor[0] + scheme.tempEqu2_cor[1]

df['AirFlow'] = df['AirFlow'] * scheme.airflow_cor[0] + scheme.airflow_cor[1]
df['WaterFlowEqu1'] = df['WaterFlowEqu1'] * scheme.waterflowEqu1_cor[0] + scheme.waterflowEqu1_cor[1]
df['WaterFlowEqu2'] = df['WaterFlowEqu2'] * scheme.waterflowEqu2_cor[0] + scheme.waterflowEqu2_cor[1]


In [ ]:
df.plot('DateTime', 'LightLX')

In [ ]:
v_state_list = list(scheme.CHANNEL_COLS.values())

chnl = 'Channel'
if not chnl in df.columns:
    df[chnl] = 0
if 'V1_state' in df.columns:
    df.loc[df.V1_state == 1, chnl] = 1
    df.loc[df.V2_state == 1, chnl] = 2
    df.loc[df.V3_state == 1, chnl] = 3
    df.loc[df.V4_state == 1, chnl] = 4
    df.loc[df.V5_state == 1, chnl] = 5
    df.loc[df.V6_state == 1, chnl] = 6
df.drop(v_state_list, inplace=True, errors='ignore')

###  Remove bad data   
List of accidental cases to filter data (see notes)

In [ ]:
filter_list = [{'date_start': '01.08.2023 00:00', 'date_stop': '01.08.2023 13:00', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro is not ready
               {'date_start': '01.08.2023 13:32', 'date_stop': '01.08.2023 13:35', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro data is zero
               {'date_start': '01.08.2023 17:00', 'date_stop': '01.08.2023 17:20', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro gas pipe fault
               {'date_start': '01.08.2023 17:19', 'date_stop': '01.08.2023 17:25', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # Picarro gas pipe fault
               {'date_start': '02.08.2023 10:54', 'date_stop': '02.08.2023 10:59', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # gas pipe fault, change of desiccant
               {'date_start': '04.08.2023 11:05', 'date_stop': '04.08.2023 11:10', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # gas pipe fault, change of desiccant
               {'date_start': '04.08.2023 12:26', 'date_stop': '04.08.2023 14:39', 'cols': ['WaterFlowEqu1'], 'fill_with': 0},       # water flow unstable
               {'date_start': '04.08.2023 14:30', 'date_stop': '04.08.2023 14:37', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # pump 'surface' disconnected
               {'date_start': '05.08.2023 02:30', 'date_stop': '05.08.2023 08:06', 'cols': ['TempEqu2'], 'fill_with': np.NaN},       # temperature sensor pulled out
               {'date_start': '06.08.2023 16:55', 'date_stop': '06.08.2023 16:59', 'cols': ['TempEqu2'], 'fill_with': np.NaN},       # temperature sensor pulled out
               {'date_start': '07.08.2023 13:23', 'date_stop': '07.08.2023 16:06', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.NaN},   # channel selector unit disfunction
               {'date_start': '07.08.2023 13:42', 'date_stop': '07.08.2023 16:56', 'cols': ['WaterFlowEqu1', 'WaterFlowEqu2'], 'fill_with': 0},  # water flow unstable
               {'date_start': '07.08.2023 16:19', 'date_stop': '07.08.2023 16:44', 'cols': ['TempEqu2'], 'fill_with': np.NaN},       # temperature sensor pulled out
               {'date_start': '08.08.2023 06:56', 'date_stop': '08.08.2023 10:56', 'cols': ['WaterFlowEqu1'], 'fill_with': 0},       # water flow unstable
               {'date_start': '08.08.2023 02:30', 'date_stop': '08.08.2023 10:00', 'cols': ['TempEqu2'], 'fill_with': np.NaN},
               {'date_start': '08.08.2023 10:43', 'date_stop': '08.08.2023 10:51', 'cols': ['TempEqu1'], 'fill_with': np.NaN},       # water flow unstable
               {'date_start': '08.08.2023 11:22', 'date_stop': '08.08.2023 11:30', 'cols': ['WaterFlowEqu2'], 'fill_with': 0},       # water flow unstable
               {'date_start': '08.08.2023 11:22', 'date_stop': '08.08.2023 11:30', 'cols': ['TempEqu2'], 'fill_with': np.NaN},
               {'date_start': '08.08.2023 14:51', 'date_stop': '08.08.2023 14:59', 'cols': ['TempEqu1'], 'fill_with': np.NaN},
              ]

for cycle in filter_list:
    for col in cycle['cols']:
        date_start = pd.to_datetime(cycle['date_start'], dayfirst=True)
        date_stop = pd.to_datetime(cycle['date_stop'], dayfirst=True)
        df.loc[(df['DateTime'] > date_start) & (df['DateTime'] < date_stop), col] = cycle['fill_with']

In [ ]:
cols = ['FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu']
for col in cols:
    df.loc[df[col] == 0, col] = np.NaN
cols = ['WaterFlowEqu1', 'WaterFlowEqu2', 'AirFlow']
for col in cols:
    df.loc[df[col] < 0, col] = np.NaN

## Flux calc

In [ ]:
df

In [ ]:
### find the moments where flux is valid to calculate
### before 02.10.2023 15:30 there was long cycle flux measure
### after this time a short cycle took place

t = pd.to_datetime('02.10.2023 15:30', dayfirst=True)
df['flag'] = (df['Channel'] == 5) 
for deep in range(50):
    df['flag'] &= (df['Channel'] == df['Channel'].shift(deep))

tempflag = (df['Channel'] == 5) & (df['DateTime'] > t) 

for deep in range(10):
    tempflag &= (df['Channel'] == df['Channel'].shift(deep))

df['flag'] |= tempflag
# set flag for 50 points skipping first 10 points  
df['flag'] = df['flag'].astype('int8')
# count flux cycles for grouping
df['fluxEvent'] = (df['flag']==0) & (df['flag'].shift(-1)!=0)
df['fluxCount'] = df['fluxEvent'].cumsum() * df['flag']
#df['flag'] |= (df['Channel'] == 5) & (df['DateTime'] > t)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(1, 1)

axs.set_title('vCO2, ppm')
axs.plot('DateTime', 'flag', '-', data=df.tail(200), c='silver')
axs.plot('DateTime', 'fluxEvent', '-', data=df.tail(200), c='red')
axs.plot('DateTime', 'fluxCount', '-', data=df.tail(200), c='blue')

In [ ]:
ch_v = 'flag'
df['cCO2chm'] = df['vCO2'][(df[chnl] == 5) & (df[ch_v] == 1)] * df['PressAir'] * bfunc.getDensity(temp=0, press=1, gas='CO2') / 1000
df['cCH4chm'] = df['vCH4'][(df[chnl] == 5) & (df[ch_v] == 1)] * df['PressAir'] * bfunc.getDensity(temp=0, press=1, gas='CH4') 
df['DateSec'] = df['DateTime'].astype('int64')//10**9
for col in ['cCO2chm', 'cCH4chm']:
    df[col] = df[col].rolling(3).mean()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1)
ax.plot('DateTime', 'cCH4chm', 'ro', linewidth=1, alpha=0.2, data=df )
ax.plot('DateTime', 'flag', 'c-', linewidth=1, alpha=0.8, data=df)
ax.set_xlim(pd.to_datetime('02.10.2023 14:00', dayfirst=True), pd.to_datetime('02.10.2023 18:00', dayfirst=True))

In [ ]:
deep = 2
df['CO2flux'] = (df['cCO2chm'].shift(-deep) - df['cCO2chm'].shift(deep)) / (df['DateSec'].shift(-deep) - df['DateSec'].shift(deep)) * 3600 / 0.81 * 320
df['CH4flux'] = (df['cCH4chm'].shift(-deep) - df['cCH4chm'].shift(deep)) / (df['DateSec'].shift(-deep) - df['DateSec'].shift(deep)) * 3600 / 0.81 * 320

In [ ]:
dft = df.copy()

In [ ]:
def ff(dff):
    dff.dropna(inplace=True)
    l = int(len(dff)/2)  # point to the middle of array
    return dff.sort_values()[l-2:l+2].mean()
    
dfr = dft['CO2flux'].groupby(dft['fluxCount']).apply(ff)
dft=dft.merge(dfr, how='left', right_on='fluxCount', left_on='fluxCount')
dfr = dft['CH4flux'].groupby(dft['fluxCount']).apply(ff)
dft=dft.merge(dfr, how='left', right_on='fluxCount', left_on='fluxCount')
dft

In [ ]:
plt.rcParams['figure.figsize'] = [17, 5]
fig, ax = plt.subplots(1)
ax.plot('DateTime', 'CO2flux_x', 'ro', linewidth=1, alpha=0.2, data=dft)
ax.plot('DateTime', 'CO2flux_y', 'go', linewidth=1, alpha=0.2, data=dft, markersize=3)
# ax.set_xlim(pd.to_datetime('02.10.2023 14:00', dayfirst=True), pd.to_datetime('08.10.2023 18:00', dayfirst=True))
ax.set_ylim(-50, 50)
ax.grid(alpha=0.2)

In [ ]:
plt.rcParams['figure.figsize'] = [17, 5]
fig, ax = plt.subplots(1)
ax.plot('DateTime', 'CH4flux_x', 'ro', linewidth=1, alpha=0.2, data=dft)
ax.plot('DateTime', 'CH4flux_y', 'go', linewidth=1, alpha=0.2, data=dft, markersize=3)
# ax.set_xlim(pd.to_datetime('02.10.2023 14:00', dayfirst=True), pd.to_datetime('08.10.2023 18:00', dayfirst=True))
ax.set_ylim(-50, 50)
ax.grid(alpha=0.2)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1)
ax.plot('DateTime', 'CH4flux_x', 'ro', linewidth=1, alpha=0.2, data=dft)
ax.plot('DateTime', 'CH4flux_y', 'go', linewidth=1, alpha=0.2, data=dft)
ax.set_xlim(pd.to_datetime('02.10.2023 14:00', dayfirst=True), pd.to_datetime('03.10.2023 18:00', dayfirst=True))
ax.set_ylim(-200, 200)

In [ ]:
df = dft.copy()

In [ ]:
filter_list = [{'date_start': '07.08.2023 17:00', 'date_stop': '07.08.2023 17:10', 'cols': ['CO2flux', 'CH4flux'], 'fill_with': np.NaN},   # Wrench is blocked
               {'date_start': '08.08.2023 01:30', 'date_stop': '08.08.2023 11:20', 'cols': ['CO2flux', 'CH4flux'], 'fill_with': np.NaN},   # Wrench is blocked
              ]

for cycle in filter_list:
    date_start = pd.to_datetime(cycle['date_start'], dayfirst=True)
    date_stop = pd.to_datetime(cycle['date_stop'], dayfirst=True)
    for col in cycle['cols']:
        df.loc[(df['DateTime'] > date_start) & (df['DateTime'] < date_stop), col] = cycle['fill_with']

In [ ]:
## do not know why is it here
df = df.resample('20S', on='DateTime').mean()
df[chnl].fillna(method='ffill', inplace=True)

In [ ]:
df['DateTime'] = df.index


In [ ]:
df['reff'] = 1
ch_v = 'Chn_valid'
df[ch_v] = 1
for deep in range(1, 7):
    df.loc[df[chnl] != df[chnl].shift(deep), ch_v] = 0
df.dropna(axis='index', subset=['reff'], inplace=True)

In [ ]:
df

In [ ]:
df['vCO2air'] = df['vCO2'][(df[chnl] == 6) & (df[ch_v] == 1)]
df['vCH4air'] = df['vCH4'][(df[chnl] == 6) & (df[ch_v] == 1)]
for col in ['vCO2air', 'vCH4air',]:
    df[col] = df[col].rolling(3).mean()

df['vCO2air'].interpolate(method='values', inplace=True)
df['vCH4air'].interpolate(method='values', inplace=True)  ## `time` method mb better
df['pCO2air'] = df['vCO2air'] * df['PressAir']  # mkatm
df['pCH4air'] = df['vCH4air'] * df['PressAir']  # mkatm

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(2, 1)

axs[0].set_title('CO2 air, ppm')
axs[0].set_ylim(150, 650)
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')
axs[0].plot(df.DateTime, df['vCO2air'], 'r-')

axs[1].set_title('CH4 air, ppm')
axs[1].set_ylim(0, 20)
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')
axs[1].plot(df.DateTime, df['vCH4air'], 'b-')
fig.savefig(saveImgPath+'cAir_vs_time.png', transparent=False)

In [ ]:
df.columns

## RECOVERY !!!

In [ ]:
df['DateSec'] = df['DateTime'].astype('int64')//10**9
df['dTSec'] = df['DateSec'] - df['DateSec'].shift(1)
dt = df['dTSec'] / 60   ## delta time, min
ch_v = 'Chn_valid'

#### recovery `CO2/CH4`, channel `1` (bottom in Oct 2023)

In [ ]:
equ_vol = scheme.equ_walltube_param['equ_vol']  # equivalent equ volume, l
equ_cap = scheme.equ_walltube_param['equ_cap']  # equilibrator capacity
wtr_flow_min = scheme.equ_walltube_param['water_flow_min'] 

In [ ]:
t_wtr = df['TempEqu1']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu1']
air_flow = df['AirFlow']

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 1) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2bot'] = cGasWtr * 1000     # mg/l
df['pCO2bot'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 1) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4bot'] = cGasWtr * 1000000000    # ng/l
df['pCH4bot'] = cGasWtr * 1000000 / solubility  # mkatm

#### recovery `CO2/CH4`, channel `2` (surface in oct 2023)

In [ ]:
t_wtr = df['TempEqu2']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu1']  # waterflow sensor at channel #2 didn't work properly

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 2) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2sur'] = cGasWtr * 1000     # mg/l
df['pCO2sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 2) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(df['TempEqu1'], 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4sur'] = cGasWtr * 1000000000    # ng/l
df['pCH4sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
## rolling average is applied inplace. Be cafelly, run this cell **ONCE**

for col in ['cCO2sur', 'cCO2bot', 'pCO2sur', 'pCO2bot', 'cCH4sur', 'pCH4sur', 'cCH4bot', 'pCH4bot']:
    df.loc[df[col] == np.inf , col] = np.nan 
    df.loc[df[col] == -np.inf , col] = np.nan
    df.loc[df[col] <= 0 , col] = np.nan
    df[col] = df[col].rolling(3, center=True).mean()

In [ ]:
df['pCH4bot'].describe()

In [ ]:
df['pCH4sur'].describe()

In [ ]:
df['pCO2'] = df['vCO2'] * df['PressAir']

plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
## ax.set_xlim(pd.to_datetime('05.06.2023 18:00:00', dayfirst=True), pd.to_datetime('05.06.2023 19:00:00', dayfirst=True))
ax.set_title('CO2 water, mkatm')
ax.set_ylim(0, 800)
ax.plot(df.DateTime, df['pCO2'], '-', c='silver')
ax.plot(df.DateTime, df['pCO2sur'], 'g-')
ax.plot(df.DateTime, df['pCO2bot'], 'r-')
ax.plot(df.DateTime, df['pCO2air'], 'c-')
fig.savefig(saveImgPath+'pCO2wtr_vs_time.png')

In [ ]:
count_recent = 30000
df['pCH4'] = df['vCH4'] * df['PressAir']
plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
ax.set_title('CH4 water, mkatm')
# ax.set_xlim(pd.to_datetime('06.06.2023 10:00:00', dayfirst=True), pd.to_datetime('06.06.2023 19:00:00', dayfirst=True))
# ax.set_ylim(0, 20)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4'], '-', c='silver')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4bot'], 'r.')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4sur'], 'g.')
fig.savefig(saveImgPath+'pCH4wtr_vs_time.png')

In [ ]:
df.columns

In [ ]:
df['hour'] = df['DateTime'].dt.hour # + df['DateTime'].dt.minute / 60
df['hour']

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1)
ax.plot(df['hour'], df['CO2flux_y'], 'ro', linewidth=1, alpha=0.2, data=df )
ax.grid(alpha=.2)
# ax.set_ylim(-100, 10)

In [ ]:
cols = ['hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        'vCO2', 'vCH4', 'vH2O', 'AirFlow', 'Channel',
        'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu', 'WaterFlowEqu1',
        'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4',
        'TempEqu1', 'TempEqu2', 'TempEqu3', 'cCO2chm', 'cCH4chm',
        'CO2flux', 'CH4flux', 'vCO2air', 'vCH4air',
        'pCO2air', 'pCH4air', 'cCO2sur', 'pCO2sur',
        'cCH4sur', 'pCH4sur', 'cCO2bot', 'pCO2bot', 'cCH4bot', 'pCH4bot',]
df.to_csv(f'{saveDataPath}BaikalOct2023.dat',
          # columns=cols,
          sep='\t',
         )

In [ ]:
func_list = ['mean', 'std']
hourly = df.resample('1H', on='DateTime').agg(func_list)

cols = ['hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        'vCO2', 'vCH4', 'vH2O',
        'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu', 'WaterFlowEqu1',
        'TempEqu1', 'TempEqu2', 'TempEqu3', 'cCO2chm', 'cCH4chm',
        'CO2flux', 'CH4flux', 'vCO2air', 'vCH4air',
        'pCO2air', 'pCH4air', 'cCO2sur', 'pCO2sur',
        'cCH4sur', 'pCH4sur', 'cCO2bot', 'pCO2bot', 'cCH4bot', 'pCH4bot',]
hourly.to_csv(f'{saveDataPath}BaikalOct2023hourly.dat',
              # columns=cols,
              sep='\t',
             )